### 抓取雪球文章

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月14日

----------

- 研究目的：获取微信公众号文章；

In [5]:
#引入包
import pandas as pd
import requests,re,json,time,os


# 自定义
import sys
sys.path.append('/home/jquser')

from dm import *
from tl import exists_file

# 数据目录
_data_path='../../Data/'
# 数据引擎
_data=Csv(_data_path)



#
base_url='https://xueqiu.com'


#时间戳转换成日期
def timestamp_date(timestamp):
    return time.strftime('%Y-%m-%d %H:%M',time.localtime(int(str(timestamp)[0:10])))


#检测文件      
def exists_xueqiu_file(user_id,data_root=_data_path):
    #cvs文件名
    data_file='%s%s.csv'%(data_root,'msg_xqwz_'+user_id)
    return exists_file(data_file)


#读取数据        
def read_xueqiu_data(user_id):
    #文件名
    data_file='msg_xqwz_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    return df


#保存数据        
def save_xueqiu_data(user_id,df):
    #文件名
    data_file='msg_xqwz_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)



#headers
header={
    "HOST":'xueqiu.com',
    'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 \
Safari/537.36 Core/1.53.4620.400 QQBrowser/9.7.13014.400'
    }

# cookies
def get_xueqiu_cookies():
    cookies_str='aliyungf_tc=AQAAAKzMsS1s2Q4AUHWuPB933Kpmjm0e; device_id=1122282b8c299527c7cb6d188fa28a44; remember=1; remember.sig=K4F3faYzmVuqC0iXIERCQf55g2Y; xq_a_token=5654fd39cb89fef36765cc9038bc5b47ad0e62ba; xq_a_token.sig=MZOhC2VaLNqrwpqJk0ONVR8kUMY; xq_r_token=01ee39b512f28c6f395e2d5fc1eeed662ade6a39; xq_r_token.sig=U3jfCcO3T4tjqSenYk1Oevgpufg; xq_is_login=1; xq_is_login.sig=J3LxgPVPUzbBg3Kee_PquUfih7Q; u=6582724674; u.sig=1g3gy3Thq5MyFexbtGy2oMIfDW0; s=ep12uiderf; bid=14636255fdee238fdda1fc0fe35fbb46_jg2y7pzt; snbim_minify=true; __utmt=1; Hm_lvt_1db88642e346389874251b5a1eded6e3=1523925825,1523925860,1523930936; Hm_lpvt_1db88642e346389874251b5a1eded6e3=1523931199; __utma=1.1796576587.1523925827.1523925827.1523930937.2; __utmb=1.6.10.1523930937; __utmc=1; __utmz=1.1523930937.2.2.utmcsr=sogou.com|utmccn=(referral)|utmcmd=referral|utmcct=/link'
    #cookies_str转换为字典格式
    cookies_str=cookies_str.split('; ')
    cookies={}
    for item in cookies_str:
        temp=item.split('=')
        cookies[temp[0]]=temp[1]  
    return cookies

#爬取微信公众号文章
#user_id：用户识别码
#cookies_str：登陆后的cookies字符串，每次登陆后都不一样，所以要手动获取
#start：更新的起始位置，用以实现断点续爬和增量更新
def get_xueqiu_link(user_id,start=0):
    #cookies_str转换为字典格式
    cookies=get_xueqiu_cookies()  
    #数据列表
    data_list=[]
    #当前页
    page=1
    #页面数
    pages=1
    #计数器
    count=1
    #捕获错误
    try:
        while page<=pages:
            url='https://xueqiu.com/statuses/original/timeline.json?user_id=%s&page=%s'%(user_id,page)
            #获取每一页文章的标题和链接地址
            r=requests.get(url,cookies=cookies,headers=header)
            column=r.json()
            #获得当前页文章列表
            item_list=column.get('list')
            #如果无数据跳出循环
            if len(item_list)==0:
                break
            #如果是首页，获取总页数、总数量、每页数量    
            if page==1:
                #总数量
                total=int(column.get('total'))  
                #如果总数量和起始数相等，则退出
                if total==start:
                    break
                #每页数量    
                item_count=int(column.get('count')) 
                #总页数
                max_page=int(column.get('maxPage')) 
                #计算实际页数
                pages=int((total-start)/item_count)
                #不足1页，算1页
                if total%item_count!=0 or pages==0 :
                    pages+=1
            #如果是最后一页，只取零头数量    
            if page==pages:    
                item_list=item_list[0:(total-start)%item_count-1]  
            #遍历当前页文章列表
            for item in item_list:
                #文章链接、标题、时间戳（时间戳转换为日期）
                data_list.append([timestamp_date(item.get('created_at')),item.get('title'),base_url+item.get('target'),'*',False])
                print(' 本次已获取：%s 篇，共 %s 篇'%(count,total),end='\r')
                #计数
                count+=1
            #页面数递减    
            page+=1
            #延时，防止被踢
            #time.sleep(1)
    #打印错误，并忽略        
    except Exception as e:
        print('错误：%s'%(e))
        pass
    #没取到数据    
    if len(data_list)==0:
        return None
    else:
        #整理数据    
        df=pd.DataFrame(data_list,columns=['date','title','link','content','sended'])    
        #返回数据
        return df
    
    
xueqiu_list=[
    {
    'user_id':'4776750571',  
    'user_name':'ETF拯救世界',    
    },
    {
    'user_id':'5941996397',  
    'user_name':'股市药丸',    
    },
]        

print(' 已准备好！')

 已准备好！


## 1.设置当前公众号

In [6]:
user_id=xueqiu_list[0]['user_id'] 
user_name=xueqiu_list[0]['user_name'] 
print(' ID   ',user_id)
print(' 名称 ',user_name)

 ID    4776750571
 名称  ETF拯救世界


## 2.获取文章链接

In [4]:
# 断点续爬数据
def update_xueqiu_data(user_id):
    #检查是否已存在数据
    if exists_xueqiu_file(user_id):
        #已存在数据
        df=read_xueqiu_data(user_id)
        #起始位置为当前记录数
        new_df=get_xueqiu_link(user_id,len(df)) 
        if new_df is None:
            print('没有新文章或更新不成功！')
        else:
            #拼接新旧数据
            df=pd.concat([new_df,df],ignore_index=True)
            #保存到cvs文件
            save_xueqiu_data(user_id,df)
            print(' 共更新：%s 篇文章！'%len(new_df))    
    else:
        #数据不存在，起始位置为0 
        df=get_xueqiu_link(user_id,0)  
        if df is None:
            print('获取不成功！')
        else:
            #保存到cvs文件
            save_xueqiu_data(user_id,df)
            print(' 共获取：%s 篇文章！'%len(df))
    
update_xueqiu_data(user_id)

错误：object of type 'NoneType' has no len()
没有新文章或更新不成功！


## 3.获取文章内容

In [4]:
# 获取文章内容
def get_xueqiu_content(user_id):
    #已获取标记
    isget=False
    #判断是否已存在文件
    if exists_xueqiu_file(user_id): 
        #读取数据
        df=read_xueqiu_data(user_id)  
        #遍历数据表
        for i in range(len(df)):
            #判断是否已获取
            if df.content.iloc[i]=='*' :
                print(' 正在获取：%s %s'%(df.date.iloc[i],df.title.iloc[i]),end='\r')
                #文章url
                url=df.link.iloc[i]
                #请求数据
                r=requests.get(url,headers=header)
                r=r.text
                #解析网页
                content=r.split('"text":')[1].split('",')[0] 
                #添加数据
                df.content.iloc[i]=content
                isget=True
                #延时5秒，防止被踢
                time.sleep(1)
    #如果获取到文章            
    if isget==True:  
        #保存数据表
        save_xueqiu_data(user_id,df) 
        print(' 文章内容获取完毕！')
    else:
        print(' 尚未获取文章列表或是没有要获取的！')
        
get_xueqiu_content(user_id)        

 文章内容获取完毕！-04-20 20:22 为什么90%的人会死在市场里


/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 4.发送邮件

In [3]:
import os

#引入自定义（jsl）库
#保存当前目录
path=os.getcwd()
#更改到用户根目录为
os.chdir("/home/jquser/")
#导入自己的库
from mail import *
#恢复当前目录
os.chdir(path)

#发送邮件
def send_xueqiu_data(user_id,user_name):
    issend=False
    if exists_xueqiu_file(user_id):
        #读取数据
        df=read_xueqiu_data(user_id)
        i=len(df)-1
        while i>=0:  
            if df['sended'].iloc[i]==False:
                #邮件主题
                subjiect='%s（雪球文章 %s）'%(df.title.iloc[i],user_name)
                #邮件正文
                message='<h2>%s</h2><p>%s</br>%s</p></br><hr />%s'%(
                    df.title.iloc[i],df.date.iloc[i],df.link.iloc[i],df.content.iloc[i])
                try:
                    #发送邮件
                    send_html_qqmail(subjiect,message)
                    #发送成功后标记
                    df['sended'].iloc[i]=True
                    issend=True
                    #进度
                    print(' 已发送：%s'%(subjiect),end='\r')
                    #延时5秒，防止被踢
                    time.sleep(5)
                except Exception as e:
                    print(' 错误：%s'%e)
                    break
            i-=1    
    if issend==True:    
        save_xueqiu_data(user_id,df) 
        print(' 发送完毕！')
    else:
        print(' 尚未获取文章列表或是没有要发送的！')
    
send_xueqiu_data(user_id,user_name)     

 发送完毕！


/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 5.查看数据表

In [8]:
df=read_xueqiu_data(user_id)
# df['sended']=True
# save_weixin_data(query,df)
df

,date,title,link,content,sended
0,2018-02-09 09:13,反鸡汤随笔,https://xueqiu.com/4776750571/101334892,"""<p><b>不要鸡汤</b></p><p>最近从美股到港股，从大蓝筹到中小创，你方蹲完我来...",True
1,2017-11-29 17:09,情绪和资金,https://xueqiu.com/4776750571/96580085,"""<p>刚才那位朋友提到的市场情绪，曾经在投资笔记中写过一些，贴出来大家共同研究吧。</p>...",True
2,2017-02-07 13:37,珍惜,https://xueqiu.com/4776750571/80856822,"""<p>美国市场早就已经过了从“散户”——“机构”为主导的阶段。</p><p><br/></...",True
3,2017-01-22 13:05,在A股赔钱其实是很奇怪的一件事,https://xueqiu.com/4776750571/80478679,"""<p>从1993年至今，就用很垃圾的上证和深成指数来说，只要长期持有超过10年，任何一年年...",True
4,2016-12-27 11:35,2016，继续感恩,https://xueqiu.com/4776750571/79438068,"""<p>是不是要写这份总结，一直没有确认。两个问题：一个是总结这个东西，终究是要写给自己，还...",True
5,2016-01-21 08:31,最大的利好和利空,https://xueqiu.com/4776750571/63930554,"""对投资来说，最大的利好就是便宜，最大的利空就是贵。<br><br>所以，不要在某个品种很便...",True
6,2016-01-19 09:31,没有股神，只有长跑赢家,https://xueqiu.com/4776750571/63805996,"""投资这件事，很多人希望找到一个股神，幻想只要跟着他投资，就能无往不利，从此日进斗金资产顺利...",True
7,2015-12-24 15:12,感恩，2015,https://xueqiu.com/4776750571/62402064,"""2015年终于要过去了。<br><br>想在雪球写个投资总结，算是对自己一年的投资经历做个...",True
8,2015-12-14 10:39,证券市场里的两种人,https://xueqiu.com/4776750571/61688888,"""我把证券市场里的人，按照风格分为两种：激进的和保守的。<br><br><br>激进的人里面...",True
9,2015-11-27 17:11,下阶段布局的重点,https://xueqiu.com/4776750571/60729568,"""该提示的风险最近这段时间天天说说了有几十篇，现在说也没有任何意义了。看到的朋友算是有缘吧。...",True
